In [14]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib

In [15]:
styles = [
    dict(selector="tr:hover",
                props=[("background", "#F1E189"), ('format',"{:.3%}")]),
    dict(selector="th", props=[("color", "#7B2869"),
                               ("border", "3px solid #7B2869"),
                               ("padding", "13px 35px"),
                               ("border-collapse", "collapse"),
                               ("background", "#F9C5D5"),
                               ("font-size", "18px"),
                               ("font-family" , 'system-ui'),
                               ('text-align', 'center'),
                               ('font-weight','bold')
                               ]),
    dict(selector="td", props=[("color", "#9D3C72"),
                               ("border", "3px solid #7B2869"),
                               ("padding", "35px 55px"),
                               ("border-collapse", "collapse"),
                               ("font-size", "15px"),
                               ("font-family" , 'system-ui'),
                               ('text-align', 'center'),
                               ("background", "#FEE3EC"),
                               ('Quantity',"{:.3%}"),
                               ('font-weight','bold')
                               ]),
    dict(selector="table", props=[
                                    ("font-family" , 'system-ui'),
                                    ("margin" , "25px auto"),
                                    ("border-collapse" , "collapse"),
                                    ("border" , "1px solid #9DE3D0"),
                                    ("border-bottom" , "2px solid #00cccc"),
                                    ('Quantity',"{:.3%}")
                                      ]),
]

In [16]:
ex5 = pd.read_csv('ex5.csv')

In [17]:
def plugIn(dataFrame):
    X = dataFrame['GPA'].to_numpy()
    Y = dataFrame['LSAT'].to_numpy()
    x_mean = X.mean()
    y_mean = Y.mean()
    n = X.shape[0]

    nom = (X - x_mean) * (Y-y_mean) 
    nom = np.sum(nom)
    dnom_x = np.sum((X - x_mean) ** 2) 
    dnom_y = np.sum((Y-y_mean) ** 2)
    dnom = np.sqrt(dnom_x * dnom_y)
    plugiIn_estimator = nom/dnom
    return plugiIn_estimator

In [19]:
plugIn_estimator = plugIn(ex5)
print(f'Plug-In Estimator for x,y correlation is {plugIn_estimator}')

Plug-In Estimator for x,y correlation is 0.776374491289407


In [20]:
B = 1000
corr_plugIn = []
for b in range(B):
    sample = ex5.sample(frac=1,replace=True)
    corr_plugIn.append(plugIn(sample))
boostrap_se = np.std(corr_plugIn)
print(f'SE for Plug-In Estimator is {boostrap_se}')

SE for Plug-In Estimator is 0.13679037099722638


In [22]:
qnorm = st.norm.ppf(0.975)
gap = qnorm * boostrap_se
CI_normal = [plugIn_estimator - gap, plugIn_estimator + gap]

lower_bound = np.quantile(corr_plugIn, 0.025)
upper_bound = np.quantile(corr_plugIn, 0.975)
CI_pivotal = [plugIn_estimator * 2 - upper_bound, plugIn_estimator * 2 + lower_bound]

CI_quantile = [lower_bound,upper_bound]

strs= [f"[ plugIn_estimator - qnorm * Boostrap SE ,"
       f" plugiIn_estimator + qnorm * Boostrap SE ]",
       f"[ plugIn_estimator * 2 - upper_quantile ,"
       f" plugiIn_estimator * 2 + lower_quantile ]",
          '[ lower_quantile , upper_quantile ]']

CI_df = pd.DataFrame.from_dict({"CI Type":['Nornal Appromixation','Pivotal','Quantile'],'_':strs,"":[CI_normal,CI_pivotal,CI_quantile]}).set_index('CI Type')



CI_df.style.set_table_styles(styles).hide(axis='columns')

CI Type,,
Nornal Appromixation,"[ plugIn_estimator - qnorm * Boostrap SE , plugiIn_estimator + qnorm * Boostrap SE ]","[0.508270290702971, 1.044478691875843]"
Pivotal,"[ plugIn_estimator * 2 - upper_quantile , plugiIn_estimator * 2 + lower_quantile ]","[0.5860367633107508, 2.0017510569280383]"
Quantile,"[ lower_quantile , upper_quantile ]","[0.4490020743492243, 0.9667122192680633]"
